In [ ]:
import pandas as pd
import numpy as np
import random

# Set seed for reproducibility
np.random.seed(42)

# Define parameters
num_items = 60
num_appraisers = 3
num_trials = 2

# Create product characteristics that affect counting difficulty
product_types = ['Medical Device', 'Drug', 'Miscellaneous']
size_categories = ['Small', 'Medium', 'Large']
packaging_types = ['Individual', 'Bulk', 'Blister Pack']
storage_locations = ['Shop Floor', 'Medical Device Storage', 'Drug Storage', 'Warehouse']

# Generate items with realistic characteristics and quantities
items = []
for i in range(1, num_items + 1):
    item_id = f'ITEM{i:03d}'
    product_type = random.choice(product_types)
    size = random.choice(size_categories)
    packaging = random.choice(packaging_types)
    location = random.choice(storage_locations)
    
    # Determine true quantity based on characteristics
    if packaging == 'Bulk':
        true_qty = np.random.randint(20, 101)  # Bulk items have higher quantities
    elif packaging == 'Blister Pack':
        true_qty = np.random.randint(5, 21)    # Blister packs typically have 5-20 units
    else:  # Individual packaging
        true_qty = np.random.randint(1, 16)    # Individual items typically have fewer units
    
    # Calculate difficulty score (1-10) based on characteristics
    difficulty = 0
    if size == 'Small': difficulty += 3
    if packaging == 'Bulk': difficulty += 4
    if product_type == 'Miscellaneous': difficulty += 2
    if location == 'Warehouse': difficulty += 1
    difficulty = max(1, min(10, difficulty))
    
    items.append({
        'Item_ID': item_id,
        'Product_Type': product_type,
        'Size': size,
        'Packaging': packaging,
        'Location': location,
        'True_Quantity': true_qty,
        'Difficulty': difficulty
    })

# Create a dataframe with item information
items_df = pd.DataFrame(items)

# Generate measurements for each appraiser and trial
appraiser_names = ['Pharmacy Staff', 'Warehouse Team', 'Team Leader']
results = []

for item_idx in range(num_items):
    item_data = items[item_idx]
    item_id = item_data['Item_ID']
    true_qty = item_data['True_Quantity']
    difficulty = item_data['Difficulty']
    standard = true_qty  # "Standard" value for Minitab
    
    for appraiser in appraiser_names:
        # Adjust error rate based on appraiser experience
        if appraiser == 'Team Leader':
            error_multiplier = 0.7  # Most experienced
        elif appraiser == 'Warehouse Team':
            error_multiplier = 1.0  # Average experience
        else:
            error_multiplier = 1.2  # Least experienced
        
        for trial in range(1, num_trials + 1):
            # Calculate error probability based on difficulty and experience
            error_prob = (difficulty / 10) * error_multiplier
            
            # Determine if error occurs and generate count
            if np.random.random() < error_prob:
                max_error = max(int(true_qty * 0.2), 1)
                error = np.random.choice([-max_error, -1, 1, max_error], p=[0.1, 0.4, 0.4, 0.1])
                counted_qty = max(0, true_qty + error)
            else:
                counted_qty = true_qty
            
            # Determine match categories
            exact_match = counted_qty == true_qty
            tolerance = max(1, int(true_qty * 0.05))
            acceptable = abs(counted_qty - true_qty) <= tolerance
            
            if counted_qty < true_qty - tolerance:
                category = "Undercount"
            elif counted_qty > true_qty + tolerance:
                category = "Overcount"
            else:
                category = "Accurate"
            
            results.append({
                'Item_ID': item_id,
                'Appraiser': appraiser,
                'Trial': trial,
                'Counted_Quantity': counted_qty,
                'True_Quantity': true_qty,
                'Standard': standard,
                'Exact_Match': exact_match,
                'Acceptable': acceptable,
                'Category': category
            })

# Create a dataframe with all measurements
results_df = pd.DataFrame(results)

# Format specifically for Minitab Attribute Agreement Analysis
minitab_data = results_df.copy()
minitab_data['Assessment'] = minitab_data['Acceptable'].astype(int)

# Create wide format with appraiser-trial combinations as columns
minitab_wide = pd.pivot_table(
    minitab_data,
    values='Assessment',
    index=['Item_ID', 'True_Quantity', 'Standard'],
    columns=['Appraiser', 'Trial'],
    aggfunc='first'
).reset_index()

# Flatten the hierarchical column names
minitab_wide.columns = [f"{col[0]}_{col[1]}" if isinstance(col, tuple) else col for col in minitab_wide.columns]

# Save the files
items_df.to_csv('inventory_items_info.csv', index=False)
results_df.to_csv('inventory_counts_detail.csv', index=False)
minitab_wide.to_csv('minitab_attribute_msa_binary.csv', index=False)

# Also generate categorical assessment for Minitab
minitab_category = pd.pivot_table(
    results_df,
    values='Category',
    index=['Item_ID', 'True_Quantity', 'Standard'],
    columns=['Appraiser', 'Trial'],
    aggfunc='first'
).reset_index()

minitab_category.columns = [f"{col[0]}_{col[1]}" if isinstance(col, tuple) else col for col in minitab_category.columns]
minitab_category.to_csv('minitab_attribute_msa_category.csv', index=False)